In [ ]:
import yfinance as yf
import pandas as pd
from darts import TimeSeries
from darts.models import ExponentialSmoothing
import plotly.graph_objs as go
from darts.utils.utils import ModelMode, SeasonalityMode
from darts.metrics import mape
import numpy as np

period = '5y'

risk_free_rate = 0.02 / 252  # Annualized risk-free rate divided by trading days
benchmark = 'SPY'

# Retrieve SPY data from yfinance
ticker = 'SPY'
data = yf.download(ticker, period=period)
# Convert to pandas DataFrame
df = pd.DataFrame(data['Close']).reset_index()
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')
df = df.asfreq('B')  # 'B' stands for business day frequency
df = df.fillna(method='ffill')  # Forward fill missing values
# Create a TimeSeries object
series = TimeSeries.from_dataframe(df, None, 'Close', fill_missing_dates=True, freq='B')
# Split data into training and test sets
#train, test = series.split_after(pd.Timestamp('2022-01-01'))\
train_percentage = 0.8# 80% for training, 20% for testing
split_index = int(len(series) * train_percentage)
train, test = series.split_after(series.time_index[split_index])
num_test_days = len(test)
def plot_forecast(series, forecast, title='SPY Price Forecast'):
    """
    Plot actual vs forecasted values using Plotly.
    
    Parameters:
        series: The actual time series data.
        forecast: The forecasted values.
        title: Title of the plot.
    """

    actual = go.Scatter(x=series.time_index, y=series.values().flatten(), mode='lines', name='Actual')
    forecasted = go.Scatter(x=forecast.time_index, y=forecast.values().flatten(), mode='lines', name='Forecast')
    layout = go.Layout(
        title=title, 
        xaxis=dict(title='Date'), 
        yaxis=dict(title='Price'),
        shapes=[
            # Highlight the training period
            dict(
                type="rect",
                xref="x",
                yref="paper",
                x0=train.time_index[0].strftime('%Y-%m-%d'),
                y0=0,
                x1=train.time_index[-1].strftime('%Y-%m-%d'),
                y1=1,
                fillcolor="LightBlue",
                opacity=0.5,
                layer="below",
                line_width=0,
            ),
            # Highlight the forecast period
            dict(
                type="rect",
                xref="x",
                yref="paper",
                x0=train.time_index[-1].strftime('%Y-%m-%d'),
                y0=0,
                x1=series.time_index[-1].strftime('%Y-%m-%d'),
                y1=1,
                fillcolor="LightCoral",
                opacity=0.5,
                layer="below",
                line_width=0,
            )
        ],
        annotations=[
            # Annotation for number of test days
            dict(
                x=train.time_index[-1] + pd.DateOffset(days=1),  # Place annotation just after the training period
                y=max(series.values().flatten()),  # Place it at the top of the plot
                xref="x",
                yref="y",
                text=f"Number of Test Days: {num_test_days}",
                showarrow=False,
                font=dict(size=12, color="black"),
                align="center",
                bgcolor="white"
            )
        ]
    )
    fig = go.Figure(data=[actual, forecasted], layout=layout)
    return fig

In [ ]:
# Initialize and train the model
model = ExponentialSmoothing()
model.fit(train)

# Make predictions
forecast = model.predict(len(test))

# Plot the results using Plotly
plot_forecast(series, forecast)

In [ ]:
# Initialize and train the model
model = ExponentialSmoothing(
    trend=ModelMode.ADDITIVE,       # Set the trend to additive
    seasonal=SeasonalityMode.NONE   # No seasonality
)
model.fit(train)

# Make predictions
forecast = model.predict(len(test))

# Plot the results using Plotly
plot_forecast(series, forecast)

In [ ]:
# Initialize and train the model
model = ExponentialSmoothing(
    trend=ModelMode.ADDITIVE,       # Set the trend to additive
    seasonal=SeasonalityMode.MULTIPLICATIVE,
    seasonal_periods=12
)
model.fit(train)

# Make predictions
forecast = model.predict(len(test))

plot_forecast(series, forecast)

In [ ]:
# Define the parameter grid
parameters = {
    'trend': [ModelMode.ADDITIVE, ModelMode.MULTIPLICATIVE],
    'seasonal': [SeasonalityMode.ADDITIVE, SeasonalityMode.MULTIPLICATIVE],
    'seasonal_periods': list(range(2, 48))
}
# Iterate through the parameter grid and find the best model
best_mape = float('inf')
best_model = None
best_params = {}

for trend in parameters['trend']:
    for seasonal in parameters['seasonal']:
        for seasonal_periods in parameters['seasonal_periods']:
            model = ExponentialSmoothing(
                trend=trend,
                seasonal=seasonal,
                seasonal_periods=seasonal_periods
            )
            model.fit(train)
            forecast = model.predict(len(test))
            error = mape(test, forecast)
            print(f'Trend: {trend}, Seasonal: {seasonal}, Seasonal Periods: {seasonal_periods} - MAPE: {error:.4f}')
            
            if error < best_mape:
                best_mape = error
                best_model = model
                best_params = {
                    'trend': trend,
                    'seasonal': seasonal,
                    'seasonal_periods': seasonal_periods
                }

print(f'Best Parameters: {best_params} - Best MAPE: {best_mape:.4f}')

# Make predictions with the best model
forecast = best_model.predict(len(test))

plot_forecast(series, forecast).show()